<a href="https://colab.research.google.com/github/spyrosviz/Injury_Prediction_MidLong_Distance_Runners/blob/main/Preprocessing/Runners_Injury_Prediction_Preprocessing_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**2.  PREPROCESSING**
**CONCATENATE THE DAILY EXCEL CONTAINING THE ADDED LOAD FEATURES WITH THE WEEKLY EXCEL SO THAT THEY CONTAIN THE SAME NUMBER OF DAYS PER ATHLETE IN DATES COLUMN**

**2.1 FIND ATHLETE IDs WHO DON'T HAVE THE SAME NUMBER OF DAYS IN DATES COLUMN**

In [ ]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
# import daily excel and get number of training days per athlete id
df_daily = pd.read_excel(r'/content/drive/MyDrive/Runners_Injury_MLproject/run_injur_with_acuteloads.xlsx',index_col=0)
df_counts_daily = df_daily.groupby('Athlete ID').count()
no_dates_daily = df_counts_daily['Date'].values

# import weekly excel and get number of training days per athlete id
df_weekly = pd.read_excel(r'/content/drive/MyDrive/Runners_Injury_MLproject/MidLongDistanceRunnersInjuryWeekly.xlsx',index_col=0)
df_counts_weekly = df_weekly.groupby('Athlete ID').count()
no_dates_weekly  = df_counts_weekly['Date'].values

# find athletes' ids where number of training days in daily excel doesn't match number of training days in weekly excel 
diff_elem = np.setdiff1d(no_dates_daily,no_dates_weekly)
ids = []
for no in no_dates_daily:
  if no in diff_elem:
    ids.append(int(np.where(no_dates_daily == no)[0]))

print(ids)



[5, 9, 19, 20, 21, 33, 39, 40, 45, 52, 53, 57, 63, 66, 69]


**2.2 CONCATENATE THE TWO EXCELS INTO ONE DATAFRAME SO THAT THEY CONTAIN THE SAME NUMBER OF DAYS PER ATHLETE IN DATES COLUMN**

In [ ]:
num_of_athletes = 74
ids_daily_drop = []
ids_weekly_drop = []

for id in range(num_of_athletes):
  df_daily_copy = df_daily[df_daily['Athlete ID']==id]
  train_days_daily = df_daily_copy['Date'].values
  
  df_weekly_copy = df_weekly[df_weekly['Athlete ID']==id]
  train_days_weekly = df_weekly_copy['Date'].values

  for day in train_days_daily:
    if day not in train_days_weekly:
      idx = int(df_daily[(df_daily['Athlete ID']==id) & (df_daily['Date']==day)].index[0])
      ids_daily_drop.append(idx)



  for day in train_days_weekly:
    if day not in train_days_daily:
      idx = df_weekly[(df_weekly['Athlete ID']==id) & (df_weekly['Date']==day)].index[0]
      ids_weekly_drop.append(idx)

df_daily.drop(ids_daily_drop,axis=0,inplace=True)
df_weekly.drop(ids_weekly_drop,axis=0,inplace=True)
print(df_daily.shape)
print(df_weekly.shape)

# Print duplicate columns from two dataframes
print(set(df_daily.columns) & set(df_weekly.columns))

# Rename different variables with same column name, reset indices and drop duplicate columns
df_weekly.rename(columns={'nr. sessions':'nr_sessions_week','nr. sessions.1':'nr_sessions.1_week','nr. sessions.2':'nr_sessions.2_week'},
                 inplace=True)
df_daily.reset_index(drop=True,inplace=True)
df_weekly.reset_index(drop=True,inplace=True) 
df_weekly.drop(['Athlete ID','injury','Date'],axis=1,inplace=True)

# Concatenate dataframes
df_concat = pd.concat([df_daily,df_weekly],axis=1)

# Replace -0.01 with 0 in some columns. For example in perceived exertion column, when it's not a training day it should have a value of 0.
# Instead the cell value has been filled with -0.01 so we replace it with 0
df_concat.replace(-0.01,0,inplace=True)
columns = df_concat.columns
print(df_concat.head)

(42680, 77)
(42680, 72)
{'nr. sessions.2', 'injury', 'nr. sessions.1', 'nr. sessions', 'Athlete ID', 'Date'}
<bound method NDFrame.head of        nr. sessions  total km  ...  rel total kms week 0_2  rel total kms week 1_2
0                 1       5.8  ...                1.378882                1.919255
1                 0       0.0  ...                1.018868                1.490566
2                 1       0.0  ...                1.018868                1.490566
3                 0       0.0  ...                1.018868                1.490566
4                 1       0.0  ...                1.361111                0.618056
...             ...       ...  ...                     ...                     ...
42675             0       0.0  ...                1.101852                0.444444
42676             1       8.0  ...                1.288889                1.222222
42677             1      14.7  ...                0.743689                0.990291
42678             1      12.2  

In [ ]:
df_concat.to_excel('concat-daily-weekly.xlsx',columns=columns)